# Plotting orbit of Jupiter  using RK 4 solver in 3D

## Konstantinos Doran SN:22007700 Date:06/03/24
Start of with just n bodies in 3D

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy import constants

In [ ]:
def force(r1,r2,m1,m2):
    """Returns the gravitational force on a mass, m1, at a point r1, due to another mass, m2, at a point r2. Assuming G is defined externally
    
    Inputs:
    r1      position of first mass as a numpy array
    r2      position of second mass as a numpy array
    m1      mass of first object
    m2      mass of second object    
    Returns:
    f_on_m1 force on mass 1 as a numpy array
    """
    # Find r_12 and its magnitude
    r12 = r2 - r1
    f_on_m1 = G*m1*m2/(np.sqrt((np.dot(r12,r12))**3))*r12
    return f_on_m1

In [ ]:
def potential(r1,r2,m1,m2):
    """Returns the gravitational potential energy of a mass, m1, at a point r1, due to another mass, m2, at a point r2.
    
    Inputs:
    r1      positions of first mass as a 2D numpy array
    r2      positions of second mass as a 2D numpy array
    m1      mass of first object
    m2      mass of second object    
    Returns:
    pot potential energy of mass 1 as a 2D numpy array
    """
    # Find r_12 and its magnitude
    r12 = r2 - r1
    pot = -G*m1*m2/np.linalg.norm(r12, axis=1)
    return pot

In [ ]:
# Set up parameters
Ndim = 3     #2D problem for now
Nsteps = 1500
Nbodies = 2
G = 1        # Appropriate units
dt = 0.01
m1 = 0.5  # Small
m2 = 1.0     # Large
masses = np.array([m1,m2])
# Initial conditions
r1 = np.array((-2/3,0,0))
v1 = np.array((0,np.sqrt(2/3),0.1))
r2 = np.array((1/3,0,0))
v2 = np.array((0,-np.sqrt(1/6),0))
y0 = np.array((r1,r2,v1,v2))

In [ ]:
def RK4_solver(fun,y0,dt,N):
    """Solve dy/dt = fun(y,t) using fourth-order RK method.
    Inputs:
    fun  f(y,t)
    y0   Initial condition array - positions and velocities of each object in the form (r1,v1,r2,v2,....)
    dt   Step size
    N    Number of steps
    Returns: two arrays of length N+1 (x and v or equivalent)
    """
    # Storage for solution
    #State vector consisting of positions and velocities of each body at each time
    # E.g   state vector at t0 = ((x1,y1), (x2,y2), (vx1,vy1), (vx2,vy2)) or simply (r1,r2,v1,v2)
    y = np.zeros((Nsteps,2*Nbodies,Ndim))
    Energies = np.zeros((Nsteps,Nbodies))
    # Initial condition
    y[0] = y0
    #print(y[0])
    t = 0
    for i in range(N-1):
        # RK4 formulae
        #print(y[i,0])
        k1 = dt*fun(y[i],t)
        k2 = dt*fun(y[i]+0.5*k1,t+0.5*dt)
        k3 = dt*fun(y[i]+0.5*k2,t+0.5*dt)
        #print(k3, y[i])
        k4 = dt*fun(y[i]+k3,t+dt)
        y[i+1]=y[i]+(k1+2*k2+2*k3+k4)/6
        t += dt
        
    return y

In [ ]:
def RHS_Orbital_motion(y,t):
    """Implements gravitational force of n bodies RHS for ODE solver
    
    Inputs:
    y_n   state vector containing r and v for each object
    t     Time (unused)
    
    Output:
    dy_n  state vector containing gradients dr and dv
    """
    statevec = np.zeros((2*Nbodies,Ndim))
    forces = np.zeros((Nbodies,Ndim))
    energy = np.zeros((Nbodies))
    #print(statevec)
    for i in range(Nbodies-1):
        for j in range(i+1,Nbodies):
            fij = force(y[i],y[j],masses[i],masses[j])
            forces[i] += fij
            forces[j] -= fij
            
    for i in range(Nbodies):
        statevec[i] = y[Nbodies+i]
        statevec[Nbodies+i] = forces[i]/masses[i]
    #f12 = force(y[0],y[1],masses[0], masses[1])      
    #f21 = -f12  
    #accel1 = f12/masses[0]
    #accel2 = f21/masses[1]
    #statevec[2] = accel1
    #statevec[3] = accel2
    #print(statevec)
    # statevec = (v1,v2,a1,a2)
    return statevec

In [ ]:
t_arr = np.linspace(0,Nsteps*dt,Nsteps)
orbits = RK4_solver(RHS_Orbital_motion,y0,dt,Nsteps)

In [ ]:
plt.plot(orbits[:,0,0], orbits[:,0,1])
plt.plot(orbits[:,1,0], orbits[:,1,1])
plt.axis("scaled")
print(orbits[0,3,:])
fig = plt.figure()
ax = fig.add_subplot(111,projection='3d')
ax.plot(orbits[:,0,0], orbits[:,0,1], orbits[:,0,2])
ax.plot(orbits[:,1,0], orbits[:,1,1], orbits[:,1,2])

In [ ]:
# Set up parameters
Ndim = 3     #2D problem for now
Nsteps = 3000
Nbodies = 3
G = 1        # Appropriate units
dt = 0.01
m1 = 1.0  
m2 = 3e-6     
m3 = 3e-8
masses = np.array([m1,m2,m3])
r12 = 1
r23 = 0.0025
v2mag = np.sqrt(G*m1*(r12-(m2*r12/(m1+m2))))/r12
v3mag = v2mag- np.sqrt((G*m2*(r23-(m3*r23/(m2+m3)))))/r23
# Initial conditions
r1 = np.array((0,0,0))
v1 = np.array((0,0,0))
r2 = np.array((r12,0,0))
v2 = np.array((0,v2mag*np.cos(-np.pi/4),v2mag*np.sin(-np.pi/4)))
r3 = np.array((r12+r23,0,0))
v3 = np.array((0,v3mag*np.cos(np.pi/3),v3mag*np.sin(2*np.pi/3)))
y0 = np.array((r1,r2,r3,v1,v2,v3))

In [ ]:
t_arr = np.linspace(0,Nsteps*dt,Nsteps)
orbits = RK4_solver(RHS_Orbital_motion,y0,dt,Nsteps)

In [ ]:
fig = plt.figure(figsize=(20,20))
ax = fig.add_subplot(121,projection='3d',)
ax.plot(orbits[:,0,0], orbits[:,0,1], orbits[:,0,2], label="$m_1$ orbit")
ax.plot(orbits[:,1,0], orbits[:,1,1], orbits[:,1,2], label="$m_2$ orbit")
ax.plot(orbits[:,2,0], orbits[:,2,1], orbits[:,2,2], label="$m_3$ orbit")
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("z")
ax.set_title("3-D plot of three body system")
ax.legend()
ax.view_init(20, 45)
ax1 = fig.add_subplot(122)
ax1.plot(orbits[:,0,0], orbits[:,0,1], label=r"$m_1$ orbit", color='y')
ax1.plot(orbits[:,1,0], orbits[:,1,1], label=r"$m_2$ orbit", color='b')
ax1.plot(orbits[:,2,0], orbits[:,2,1], label=r"$m_3$ orbit", color='r')
ax1.plot(orbits[0,0,0], orbits[0,0,1], "o", color='y',label=r"$m_1$")
ax1.plot(orbits[0,1,0], orbits[0,1,1], "o" , color='b', label=r"$m_2$")
ax1.plot(orbits[0,2,0], orbits[0,2,1], "o" , color='r', label=r"$m_3$")
ax1.axis('scaled')
ax1.set_xlabel("x")
ax1.set_ylabel("y")
ax1.set_title("x-y plot of three body system")
ax1.legend()

In [ ]:
fig.savefig("3d rk4 3 body orbit.pdf")

In [ ]:
fig3 = plt.figure(figsize=(26,5))
ax1 = fig3.add_subplot(1,2,1)
ax1.plot(orbits[:,0,0], orbits[:,0,1])
ax1.plot(orbits[:,1,0], orbits[:,1,1])
ax1.plot(orbits[:,2,0], orbits[:,2,1])
ax1.axis('scaled')
ax1.set_xlabel("x")
ax1.set_ylabel("y")
#ax1.set_title("Three bodies using velocity Verlet")
ax2 = fig3.add_subplot(1,3,2)
ax2.plot(t_arr,orbits[:,1,0], label="x")
ax2.plot(t_arr,orbits[:,1,1], label="y")
ax2.plot(t_arr,orbits[:,1,2], label="z")
ax2.set_xlabel("time")
ax2.set_ylabel("position")
ax2.set_title("Position of mass 2 with respect to time")
ax2.legend()
ax3 = fig3.add_subplot(1,3,3)
ax3.plot(t_arr,orbits[:,2,0], label="x")
ax3.plot(t_arr,orbits[:,2,1], label="y")
ax3.plot(t_arr,orbits[:,2,2], label="z")
ax3.set_xlabel("time")
ax3.set_ylabel("position")
ax3.set_title("Position of mass 3 with respect to time")
ax3.legend()    

In [ ]:
def totalenergy(orbits):
    """
    """
    TotE = np.zeros((Nsteps))
    KE = np.zeros((Nbodies,Nsteps))
    PE = np.zeros((Nbodies,Nsteps))
    for i in range(Nbodies):
        KE[i] = 0.5*masses[i]*np.linalg.norm(orbits[:,Nbodies+i], axis=1)*np.linalg.norm(orbits[:,Nbodies+i], axis=1)
        for j in range(Nbodies):
            if j !=i:
                PE[i] += potential(orbits[:,i], orbits[:,j], masses[i], masses[j])
    TotE = KE + PE
    return TotE, KE, PE

In [ ]:
def totalangmomentum(orbits):
    """"""
    angularmom = np.zeros((Nbodies,Nsteps, Ndim))
    for i in range(Nbodies):
        angularmom[i] = np.cross(orbits[:,i], orbits[:,Nbodies+i])*masses[i]
        #print(angularmom[i])
    totmomentum = np.sum(angularmom,axis=0)
    #print(totmomentum)
    return totmomentum

In [ ]:
Energy, KE, PE = totalenergy(orbits)
print(Energy)
#plt.plot(t_arr, Energy[0])
#plt.plot(t_arr, Energy[1])
fig = plt.figure(figsize=(5,5))
ax = fig.add_subplot(111)
ax.plot(t_arr, Energy[0]+Energy[1]+Energy[2])
ax.set_xlabel("Time")
ax.set_ylabel("E")
ax.set_title("Total Energy over time")

In [ ]:
fig.savefig("RK4 3 body total energy 3D.pdf")

In [ ]:
L = totalangmomentum(orbits)
angfig = plt.figure(figsize=(10,6), tight_layout=True)
#print(L[0,1])
ax1 = angfig.add_subplot(131)
ax1.plot(t_arr, L[:,0])
ax1.set_xlabel("Time")
ax1.set_ylabel(r"$L_x$")
ax2 = angfig.add_subplot(132)
ax2.plot(t_arr, L[:,1])
ax2.set_xlabel("Time")
ax2.set_ylabel(r"$L_y$")
ax3 = angfig.add_subplot(133)
ax3.plot(t_arr, L[:,2])
ax3.set_xlabel("Time")
ax3.set_ylabel(r"$L_z$")
angfig.suptitle("Total Angular Momentum of system")

In [ ]:
angfig.savefig("RK4 3 body ang momentum 3D.pdf")

Add reference to values

In [ ]:
# Set up parameters
# Values taken from https://ssd.jpl.nasa.gov/horizons/app.html#/ with date set to 06-03-24
Ndim = 3     #3D problem for now
Nsteps = 600000
Nbodies = 2
G = constants.G        # Appropriate units
dt = 60*30    # s
mSun = 1988500 * 10**24 # kg
mJup = 189818722 * 10**19     # kg
masses = np.array([mSun,mJup])
# Initial conditions
r1 = np.array((-1.142389309034058E+06,-5.009058423486830E+05,3.095836895255174E+04))*1000 # m
v1 = np.array((9.244118454845832E-03,-1.136017714077767E-02,-1.036700033153707E-04))*1000 # m/s
r2 = np.array((4.656502915513301E+08,5.834910884460863E+08,-1.283856850003496E+07))*1000 # m
v2 = np.array((-1.035714698991324E+01,8.770443233178732E+00,1.953703646066063E-01))*1000 # m/s
y0 = np.array((r1,r2,v1,v2))
print(G)

In [ ]:
t_arr = np.linspace(0,Nsteps*dt,Nsteps)
orbits2 = RK4_solver(RHS_Orbital_motion,y0,dt,Nsteps)

Add different 2D viewpoints

In [ ]:
fig = plt.figure(figsize=(20,20))
ax = fig.add_subplot(121,projection='3d',)
ax.plot(orbits2[:,0,0], orbits2[:,0,1], orbits2[:,0,2], label="Sun's orbit")
ax.plot(orbits2[:,1,0], orbits2[:,1,1], orbits2[:,1,2], label="Jupiter's orbit")
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("z")
ax.set_title("3-D plot of Jupiter's orbit")
ax.legend()
ax.view_init(20, 45)
ax1 = fig.add_subplot(122)
ax1.plot(orbits2[:,0,0], orbits2[:,0,1], label="Sun's orbit", color='y')
ax1.plot(orbits2[:,1,0], orbits2[:,1,1], label="Jupiter's orbit", color='b')
ax1.plot(orbits2[0,0,0], orbits2[0,0,1], "o", color='y',label="Sun")
ax1.plot(orbits2[0,1,0], orbits2[0,1,1], "o" , color='b', label="Jupiter")
ax1.axis('scaled')
ax1.set_xlabel("x")
ax1.set_ylabel("y")
ax1.set_title("x-y plot of Jupiter's orbit")
ax1.legend()

In [ ]:
fig.savefig("Jupiter-Sun orbit.pdf")

In [ ]:
plt.plot(t_arr,orbits2[:,1,0] )
plt.plot(t_arr,orbits2[:,1,1] )
plt.plot(t_arr,orbits2[:,1,2] )
#plt.ylim(6e+11,8.1e+11)

In [ ]:
Energy, KE, PE = totalenergy(orbits2)
print(Energy)
#plt.plot(t_arr, Energy[0])
#plt.plot(t_arr, Energy[1])
fig = plt.figure(figsize=(5,5))
ax = fig.add_subplot(111)
ax.plot(t_arr, Energy[0]+Energy[1])
ax.set_xlabel("Time")
ax.set_ylabel("E")
ax.set_title("Total Energy over time")

In [ ]:
fig.savefig("Jupiter-Sun energy.pdf")

In [ ]:
L = totalangmomentum(orbits2)
angfig = plt.figure(figsize=(10,6), tight_layout=True)
#print(L[0,1])
ax1 = angfig.add_subplot(131)
ax1.plot(t_arr, L[:,0])
ax1.set_xlabel("Time")
ax1.set_ylabel(r"$L_x$")
ax2 = angfig.add_subplot(132)
ax2.plot(t_arr, L[:,1])
ax2.set_xlabel("Time")
ax2.set_ylabel(r"$L_y$")
ax3 = angfig.add_subplot(133)
ax3.plot(t_arr, L[:,2])
ax3.set_xlabel("Time")
ax3.set_ylabel(r"$L_z$")
angfig.suptitle("Total Angular Momentum of system")

In [ ]:
angfig.savefig("Jupiter-Sun ang momentum.pdf")

RK4 struggling to conserve angular momentum of system, particlularly in 3D where always 1 axis seems to fluctuate considerably

In [ ]:
# Set up parameters
# Position and velocity values taken from https://ssd.jpl.nasa.gov/horizons/app.html#/ with date set to 06-03-24
Ndim = 3     #3D problem for now
Nsteps = 600000
Nbodies = 3
G = constants.G        # Appropriate units
dt = 60*30            # s
mSun = 1988500 * 10**24 # kg
mJup = 189818722 * 10**19     # kg
mGanymede = 1.482* 10**23     # kg from https://doi.org/10.1126%2Fscience.286.5437.77
masses = np.array([mSun,mJup,mGanymede])
# Initial conditions
r1 = np.array((-1.142389309034058E+06,-5.009058423486830E+05,3.095836895255174E+04))*1000 # m
v1 = np.array((9.244118454845832E-03,-1.136017714077767E-02,-1.036700033153707E-04))*1000 # m/s
r2 = np.array((4.656502915513301E+08,5.834910884460863E+08,-1.283856850003496E+07))*1000 # m
v2 = np.array((-1.035714698991324E+01,8.770443233178732E+00,1.953703646066063E-01))*1000 # m/s
r3 = np.array((4.663298235072411E+08,5.826676629609896E+08 ,-1.286038673101431E+07))* 1000 # m
v3 = np.array((-1.954342380671196E+00, 1.571157393854946E+01, 5.784998944007400E-01))*1000 # m/s
y0 = np.array((r1,r2,r3,v1,v2,v3))

In [ ]:
t_arr = np.linspace(0,Nsteps*dt,Nsteps)
orbits3 = RK4_solver(RHS_Orbital_motion,y0,dt,Nsteps)

In [ ]:
fig = plt.figure(figsize=(20,20))
ax = fig.add_subplot(121,projection='3d',)
ax.plot(orbits3[:,0,0], orbits3[:,0,1], orbits3[:,0,2], label="Sun's orbit")
ax.plot(orbits3[:,1,0], orbits3[:,1,1], orbits3[:,1,2], label="Jupiter's orbit")
ax.plot(orbits3[:,2,0], orbits3[:,2,1], orbits3[:,2,2], label="Ganymede's orbit")
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("z")
ax.set_title("3-D plot of Jupiter's orbit")
ax.legend()
ax.view_init(20, 45)
ax1 = fig.add_subplot(122)
ax1.plot(orbits3[:,0,0], orbits3[:,0,1], label="Sun's orbit")
ax1.plot(orbits3[:,1,0], orbits3[:,1,1], label="Jupiter's orbit")
ax1.plot(orbits3[:,2,0], orbits3[:,2,1], label="Ganymede's orbit")
ax1.plot(orbits3[0,0,0], orbits3[0,0,1], "o", color='b')
ax1.plot(orbits3[0,1,0], orbits3[0,1,1], "o" , color='b')
ax1.plot(orbits3[0,2,0], orbits3[0,2,1],  "o" , color='b')
ax1.axis('scaled')
ax1.set_xlabel("x")
ax1.set_ylabel("y")
ax1.set_title("x-y plot of Jupiter's orbit")
ax1.legend()

In [ ]:
ganorbit = orbits3[:,2] - orbits3[:,1]
fig = plt.figure(figsize=(20,20))
ax = fig.add_subplot(121,projection='3d',)
ax.plot(ganorbit[:,0],ganorbit[:,1],ganorbit[:,2], label="Ganymede's orbit")
ax.plot(0,0,0,   "o" , color='b', label="Jupiter")
ax.plot(ganorbit[0,0],ganorbit[0,1],ganorbit[0,2],  "o" , color='r')
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("z")
ax.set_title("3-D plot of Ganymede's orbit")
ax.legend()
ax.view_init(20, 45)
ax1 = fig.add_subplot(122)
ax1.plot(ganorbit[:,0],ganorbit[:,1], label="Ganymede's orbit")
ax1.plot(0, 0,   "o" , color='b', label="Jupiter")
ax1.plot(ganorbit[0,0],ganorbit[0,1],  "o" , color='r')
ax1.axis('scaled')
ax1.set_xlabel("x")
ax1.set_ylabel("y")
ax1.set_title("x-y plot of Ganymede's orbit")
ax1.legend()

In [ ]:
fig.savefig("Ganymede orbit.pdf")

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(121,projection='3d',)
ax.view_init(0,90)
#ax.plot(orbits[:,1,0], orbits[:,1,1], orbits[:,1,2])
ax.plot(ganorbit[:10000,0], ganorbit[:10000,1], ganorbit[:10000,2])
ax2 = fig.add_subplot(122)
#ax.plot(orbits[:,1,0], orbits[:,1,1], orbits[:,1,2])
ax2.plot(ganorbit[:10000,0], ganorbit[:10000,1])

Effect of ganymede on orbit of Jupiter:


In [ ]:
orbitdiff = orbits3[:,1]-orbits2[:,1]
#plt.plot(orbitdiff)
fig3 = plt.figure(figsize=(28,5))
ax1 = fig3.add_subplot(1,3,1)
#ax1.plot(orbits[:,0,0], orbits[:,0,1])
#ax1.plot(orbits[:,1,0], orbits[:,1,1])
ax1.plot(t_arr, orbitdiff[:,0])
ax1.set_xlabel("Time")
ax1.set_ylabel("x")
ax1.set_title("Change in Jupiter's orbit in x direction")
ax1.grid()
ax2 = fig3.add_subplot(1,3,2)
ax2.plot(t_arr, orbitdiff[:,1])
ax2.set_xlabel("Time")
ax2.set_ylabel("y")
ax2.set_title("Change in Jupiter's orbit in y direction")
ax2.legend()
ax2.grid()
ax3 = fig3.add_subplot(1,3,3)
ax3.plot(t_arr, orbitdiff[:,2])
ax3.set_xlabel("Time")
ax3.set_ylabel("z")
ax3.set_title("Change in Jupiter's orbit in z direction")
ax3.legend()
ax3.grid()    

In [ ]:
fig3.savefig("Ganymede effect on Jupiter.pdf")

In [ ]:
Energy, KE, PE = totalenergy(orbits3)
print(Energy)
#plt.plot(t_arr, Energy[0])
#plt.plot(t_arr, Energy[1])
fig = plt.figure(figsize=(5,5))
ax = fig.add_subplot(111)
ax.plot(t_arr, Energy[0]+Energy[1]+Energy[2])
ax.set_xlabel("Time")
ax.set_ylabel("E")
ax.set_title("Total Energy over time")

In [ ]:
fig.savefig("Ganymede energy.pdf")

In [ ]:
L = totalangmomentum(orbits3)
angfig = plt.figure(figsize=(10,6), tight_layout=True)
#print(L[0,1])
ax1 = angfig.add_subplot(131)
ax1.plot(t_arr, L[:,0])
ax1.set_xlabel("Time")
ax1.set_ylabel(r"$L_x$")
ax2 = angfig.add_subplot(132)
ax2.plot(t_arr, L[:,1])
ax2.set_xlabel("Time")
ax2.set_ylabel(r"$L_y$")
ax3 = angfig.add_subplot(133)
ax3.plot(t_arr, L[:,2])
ax3.set_xlabel("Time")
ax3.set_ylabel(r"$L_z$")
angfig.suptitle("Total Angular Momentum of system")

In [ ]:
angfig.savefig("Ganymede ang momentum.pdf")

In [ ]:
# Set up parameters
# Position and velocity values taken from https://ssd.jpl.nasa.gov/horizons/app.html#/ with date set to 06-03-24
Ndim = 3     #3D problem for now
Nsteps = 600000
Nbodies = 4
G = constants.G        # Appropriate units
dt = 60*30            # s
mSun = 1988500 * 10**24 # kg
mJup = 189818722 * 10**19     # kg
mGanymede = 1.482* 10**23     # kg from https://doi.org/10.1126%2Fscience.286.5437.77
mCallisto = 1.076* 10**23     # kg from https://doi.org/10.1126%2Fscience.286.5437.77
masses = np.array([mSun,mJup,mGanymede,mCallisto])
# Initial conditions
r1 = np.array((-1.142389309034058E+06,-5.009058423486830E+05,3.095836895255174E+04))*1000 # m
v1 = np.array((9.244118454845832E-03,-1.136017714077767E-02,-1.036700033153707E-04))*1000 # m/s
r2 = np.array((4.656502915513301E+08,5.834910884460863E+08,-1.283856850003496E+07))*1000 # m
v2 = np.array((-1.035714698991324E+01,8.770443233178732E+00,1.953703646066063E-01))*1000 # m/s
r3 = np.array((4.663298235072411E+08,5.826676629609896E+08 ,-1.286038673101431E+07))* 1000 # m
v3 = np.array((-1.954342380671196E+00, 1.571157393854946E+01, 5.784998944007400E-01))*1000 # m/s
r4 = np.array((4.646133092078045E+08,  5.850685313774981E+08, -1.280301715879852E+07))* 1000 # m
v4 = np.array((-1.722217212361970E+01,  4.327917320305715E+00, -3.556385749064983E-02))*1000 # m/s
y0 = np.array((r1,r2,r3,r4,v1,v2,v3,v4))

In [ ]:
t_arr = np.linspace(0,Nsteps*dt,Nsteps)
orbits4 = RK4_solver(RHS_Orbital_motion,y0,dt,Nsteps)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111,projection='3d',)
ax.view_init(45,45)
ax.plot(orbits4[:,0,0], orbits4[:,0,1], orbits4[:,0,2])
ax.plot(orbits4[:,1,0], orbits4[:,1,1], orbits4[:,1,2])
ax.plot(orbits4[:,2,0], orbits4[:,2,1], orbits4[:,2,2])
ax.plot(orbits4[:,3,0], orbits4[:,3,1], orbits4[:,3,2])

In [ ]:
ganorbit = orbits4[:,2] - orbits4[:,1]
callorbit = orbits4[:,3] - orbits4[:,1]
plt.plot(callorbit[:500])

In [ ]:
fig = plt.figure(figsize=(20,20))
ax = fig.add_subplot(121,projection='3d',)
ax.plot(ganorbit[:,0],ganorbit[:,1],ganorbit[:,2], label="Ganymede's orbit")
ax.plot(callorbit[:,0],callorbit[:,1],callorbit[:,2], label="Callisto's orbit")
ax.plot(0,0,0,   "o" , color='b', label="Jupiter")
ax.plot(ganorbit[0,0],ganorbit[0,1],ganorbit[0,2],  "o" , color='r')
ax.plot(callorbit[0,0],callorbit[0,1],callorbit[0,2],  "o" , color='r')
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("z")
ax.set_title("3-D plot of Callisto's orbit")
ax.legend()
ax.view_init(20, 45)
ax1 = fig.add_subplot(122)
ax1.plot(ganorbit[:,0],ganorbit[:,1], label="Ganymede's orbit")
ax1.plot(callorbit[:,0],callorbit[:,1], label="Callisto's orbit")
ax1.plot(0, 0,   "o" , color='b', label="Jupiter")
ax1.plot(ganorbit[0,0],ganorbit[0,1],  "o" , color='r')
ax1.plot(callorbit[0,0],callorbit[0,1],   "o" , color='r')
ax1.axis('scaled')
ax1.set_xlabel("x")
ax1.set_ylabel("y")
ax1.set_title("x-y plot of Callisto's orbit")
ax1.legend()

In [ ]:
fig.savefig("Callisto orbit.pdf")

In [ ]:
orbitdiff43 = orbits4[:,1]-orbits3[:,1]
orbitdiff42 = orbits4[:,1]-orbits2[:,1]
#plt.plot(orbitdiff42)
fig3 = plt.figure(figsize=(28,5))
ax1 = fig3.add_subplot(1,3,1)
#ax1.plot(orbits[:,0,0], orbits[:,0,1])
#ax1.plot(orbits[:,1,0], orbits[:,1,1])
ax1.plot(t_arr, orbitdiff42[:,0])
ax1.set_xlabel("Time")
ax1.set_ylabel("x")
ax1.set_title("Change in Jupiter's orbit in x direction")
ax1.grid()
ax2 = fig3.add_subplot(1,3,2)
ax2.plot(t_arr, orbitdiff42[:,1])
ax2.set_xlabel("Time")
ax2.set_ylabel("y")
ax2.set_title("Change in Jupiter's orbit in y direction")
ax2.legend()
ax2.grid()
ax3 = fig3.add_subplot(1,3,3)
ax3.plot(t_arr, orbitdiff42[:,2])
ax3.set_xlabel("Time")
ax3.set_ylabel("z")
ax3.set_title("Change in Jupiter's orbit in z direction")
ax3.legend()
ax3.grid()    

In [ ]:
fig3.savefig("2 moons effect on Jupiter.pdf")

In [ ]:
plt.plot(orbitdiff43)

In [ ]:
Energy, KE, PE = totalenergy(orbits4)
fig = plt.figure(figsize=(5,5))
ax = fig.add_subplot(111)
ax.plot(t_arr, Energy[0]+Energy[1]+Energy[2]+Energy[3])
ax.set_xlabel("Time")
ax.set_ylabel("E")
ax.set_title("Total Energy over time")

In [ ]:
fig.savefig("2 moon energy.pdf")

In [ ]:
L = totalangmomentum(orbits4)
angfig = plt.figure(figsize=(10,6), tight_layout=True)
#print(L[0,1])
ax1 = angfig.add_subplot(131)
ax1.plot(t_arr, L[:,0])
ax1.set_xlabel("Time")
ax1.set_ylabel(r"$L_x$")
ax2 = angfig.add_subplot(132)
ax2.plot(t_arr, L[:,1])
ax2.set_xlabel("Time")
ax2.set_ylabel(r"$L_y$")
ax3 = angfig.add_subplot(133)
ax3.plot(t_arr, L[:,2])
ax3.set_xlabel("Time")
ax3.set_ylabel(r"$L_z$")
angfig.suptitle("Total Angular Momentum of system")

In [ ]:
angfig.savefig("2 moon ang momentum.pdf")

In [ ]:
# Set up parameters
# Position and velocity values taken from https://ssd.jpl.nasa.gov/horizons/app.html#/ with date set to 06-03-24
Ndim = 3     #3D problem for now
Nsteps = 600000
Nbodies = 5
G = constants.G        # Appropriate units
dt = 60*30            # s
mSun = 1988500 * 10**24 # kg
mJup = 189818722 * 10**19     # kg
mGanymede = 1.482* 10**23     # kg from https://doi.org/10.1126%2Fscience.286.5437.77
mCallisto = 1.076* 10**23     # kg from https://doi.org/10.1126%2Fscience.286.5437.77
mIo = 0.8932*10**23           # kg from https://doi.org/10.1126%2Fscience.286.5437.77
masses = np.array([mSun,mJup,mGanymede,mCallisto,mIo])
# Initial conditions
r1 = np.array((-1.142389309034058E+06,-5.009058423486830E+05,3.095836895255174E+04))*1000 # m
v1 = np.array((9.244118454845832E-03,-1.136017714077767E-02,-1.036700033153707E-04))*1000 # m/s
r2 = np.array((4.656502915513301E+08,5.834910884460863E+08,-1.283856850003496E+07))*1000 # m
v2 = np.array((-1.035714698991324E+01,8.770443233178732E+00,1.953703646066063E-01))*1000 # m/s
r3 = np.array((4.663298235072411E+08,5.826676629609896E+08 ,-1.286038673101431E+07))* 1000 # m
v3 = np.array((-1.954342380671196E+00, 1.571157393854946E+01, 5.784998944007400E-01))*1000 # m/s
r4 = np.array((4.646133092078045E+08,  5.850685313774981E+08, -1.280301715879852E+07))* 1000 # m
v4 = np.array((-1.722217212361970E+01,  4.327917320305715E+00, -3.556385749064983E-02))*1000 # m/s
r5 = np.array((4.652368585181826E+08,  5.835828494477390E+08, -1.284137882235295E+07))* 1000 # m
v5 = np.array((-1.410261135257421E+01, -8.071639586690274E+00, -4.515334237343565E-01))*1000 # m/s
y0 = np.array((r1,r2,r3,r4,r5,v1,v2,v3,v4,v5))

In [ ]:
t_arr = np.linspace(0,Nsteps*dt,Nsteps)
orbits5 = RK4_solver(RHS_Orbital_motion,y0,dt,Nsteps)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111,projection='3d',)
ax.view_init(0,45)
ax.plot(orbits5[:,0,0], orbits5[:,0,1], orbits5[:,0,2])
ax.plot(orbits5[:,1,0], orbits5[:,1,1], orbits5[:,1,2])
ax.plot(orbits5[:,2,0], orbits5[:,2,1], orbits5[:,2,2], label="Ganymede")
ax.plot(orbits5[:,3,0], orbits5[:,3,1], orbits5[:,3,2], label="Callisto")
ax.plot(orbits5[:,4,0], orbits5[:,4,1], orbits5[:,4,2], label="Io")
ax.legend()

In [ ]:
ganorbit = orbits5[:,2] - orbits5[:,1]
callorbit = orbits5[:,3] - orbits5[:,1]
Ioorbit = orbits5[:,4] - orbits5[:,1]
plt.plot(Ioorbit[:500])


In [ ]:
fig = plt.figure(figsize=(20,20))
ax = fig.add_subplot(121,projection='3d',)
ax.plot(ganorbit[:,0],ganorbit[:,1],ganorbit[:,2], label="Ganymede's orbit")
ax.plot(callorbit[:,0],callorbit[:,1],callorbit[:,2], label="Callisto's orbit")
ax.plot(Ioorbit[:,0],Ioorbit[:,1],Ioorbit[:,2],label="Io's orbit")
ax.plot(0,0,0,   "o" , color='b', label="Jupiter")
ax.plot(ganorbit[0,0],ganorbit[0,1],ganorbit[0,2],  "o" , color='r')
ax.plot(callorbit[0,0],callorbit[0,1],callorbit[0,2],  "o" , color='r')
ax.plot(Ioorbit[0,0],Ioorbit[0,1],Ioorbit[0,2],  "o" , color='r')
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("z")
ax.set_title("3-D plot of Io's orbit")
ax.legend()
ax.view_init(20, 45)
ax1 = fig.add_subplot(122)
ax1.plot(ganorbit[:,0],ganorbit[:,1], label="Ganymede's orbit")
ax1.plot(callorbit[:,0],callorbit[:,1], label="Callisto's orbit")
ax1.plot(Ioorbit[:,0],Ioorbit[:,1],label="Io's orbit")
ax1.plot(0, 0,   "o" , color='b', label="Jupiter")
ax1.plot(ganorbit[0,0],ganorbit[0,1],  "o" , color='r')
ax1.plot(callorbit[0,0],callorbit[0,1],   "o" , color='r')
ax1.plot(Ioorbit[0,0],Ioorbit[0,1],  "o" , color='r')
ax1.axis('scaled')
ax1.set_xlabel("x")
ax1.set_ylabel("y")
ax1.set_title("x-y plot of Io's orbit")
ax1.legend()

In [ ]:
fig.savefig("Io orbit.pdf")

With Io orbiting so close to Jupiter, it requires the time step to be even smaller so that the numerical integration remains accurate. However, this comes with doubling the simulation calculation time


In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111,projection='3d',)
ax.view_init(0,45)
#ax.plot(orbits[:,1,0], orbits[:,1,1], orbits[:,1,2])
ax.plot(ganorbit[:1000,0], ganorbit[:1000,1], ganorbit[:1000,2], label="Ganymede")
ax.plot(callorbit[:1000,0], callorbit[:1000,1], callorbit[:1000,2], label="Callisto")
ax.plot(Ioorbit[:1000,0], Ioorbit[:1000,1], Ioorbit[:1000,2], label="Io")
ax.legend()

In [ ]:
orbitdiff54 = orbits5[:,1]-orbits4[:,1]
orbitdiff53 = orbits5[:,1]-orbits3[:,1]
orbitdiff52 = orbits5[:,1]-orbits2[:,1]
#plt.plot(orbitdiff52)
fig3 = plt.figure(figsize=(28,5))
ax1 = fig3.add_subplot(1,3,1)
#ax1.plot(orbits[:,0,0], orbits[:,0,1])
#ax1.plot(orbits[:,1,0], orbits[:,1,1])
ax1.plot(t_arr, orbitdiff52[:,0])
ax1.set_xlabel("Time")
ax1.set_ylabel("x")
ax1.grid()
ax1.set_title("Change in Jupiter's orbit in x direction")
ax2 = fig3.add_subplot(1,3,2)
ax2.plot(t_arr, orbitdiff52[:,1])
ax2.set_xlabel("Time")
ax2.set_ylabel("y")
ax2.set_title("Change in Jupiter's orbit in y direction")
ax2.legend()
ax2.grid()
ax3 = fig3.add_subplot(1,3,3)
ax3.plot(t_arr, orbitdiff52[:,2])
ax3.set_xlabel("Time")
ax3.set_ylabel("z")
ax3.set_title("Change in Jupiter's orbit in z direction")
ax3.legend()    
ax3.grid()

In [ ]:
fig3.savefig("3 moons effect on Jupiter.pdf")

In [ ]:
Energy, KE, PE = totalenergy(orbits5)
#print(Energy)
#plt.plot(t_arr, Energy[0])
#plt.plot(t_arr, Energy[1])
fig = plt.figure(figsize=(5,5))
ax = fig.add_subplot(111)
ax.plot(t_arr, Energy[0]+Energy[1]+Energy[2]+Energy[3]+Energy[4])
ax.set_xlabel("Time")
ax.set_ylabel("E")
ax.set_title("Total Energy over time")

In [ ]:
fig.savefig("3 moon energy.pdf")

In [ ]:
L = totalangmomentum(orbits5)
angfig = plt.figure(figsize=(10,6), tight_layout=True)
#print(L[0,1])
ax1 = angfig.add_subplot(131)
ax1.plot(t_arr, L[:,0])
ax1.set_xlabel("Time")
ax1.set_ylabel(r"$L_x$")
ax2 = angfig.add_subplot(132)
ax2.plot(t_arr, L[:,1])
ax2.set_xlabel("Time")
ax2.set_ylabel(r"$L_y$")
ax3 = angfig.add_subplot(133)
ax3.plot(t_arr, L[:,2])
ax3.set_xlabel("Time")
ax3.set_ylabel(r"$L_z$")
angfig.suptitle("Total Angular Momentum of system")

In [ ]:
angfig.savefig("3 moon ang momentum.pdf")

In [ ]:
# Set up parameters
# Position and velocity values taken from https://ssd.jpl.nasa.gov/horizons/app.html#/ with date set to 06-03-24
Ndim = 3     #3D problem for now
Nsteps = 600000
Nbodies = 6
G = constants.G        # Appropriate units
dt = 60*30            # s
mSun = 1988500 * 10**24 # kg
mJup = 189818722 * 10**19     # kg
mGanymede = 1.482* 10**23     # kg from https://doi.org/10.1126%2Fscience.286.5437.77
mCallisto = 1.076* 10**23     # kg from https://doi.org/10.1126%2Fscience.286.5437.77
mIo = 0.8932*10**23           # kg from https://doi.org/10.1126%2Fscience.286.5437.77
mEuropa = 0.480*10**23        # kg from https://doi.org/10.1126%2Fscience.286.5437.77  
masses = np.array([mSun,mJup,mGanymede,mCallisto,mIo,mEuropa])
# Initial conditions
r1 = np.array((-1.142389309034058E+06,-5.009058423486830E+05,3.095836895255174E+04))*1000 # m
v1 = np.array((9.244118454845832E-03,-1.136017714077767E-02,-1.036700033153707E-04))*1000 # m/s
r2 = np.array((4.656502915513301E+08,5.834910884460863E+08,-1.283856850003496E+07))*1000 # m
v2 = np.array((-1.035714698991324E+01,8.770443233178732E+00,1.953703646066063E-01))*1000 # m/s
r3 = np.array((4.663298235072411E+08,5.826676629609896E+08 ,-1.286038673101431E+07))* 1000 # m
v3 = np.array((-1.954342380671196E+00, 1.571157393854946E+01, 5.784998944007400E-01))*1000 # m/s
r4 = np.array((4.646133092078045E+08,  5.850685313774981E+08, -1.280301715879852E+07))* 1000 # m
v4 = np.array((-1.722217212361970E+01,  4.327917320305715E+00, -3.556385749064983E-02))*1000 # m/s
r5 = np.array((4.652368585181826E+08,  5.835828494477390E+08, -1.284137882235295E+07))* 1000 # m
v5 = np.array((-1.410261135257421E+01, -8.071639586690274E+00, -4.515334237343565E-01))*1000 # m/s
r6 = np.array((4.657534133009982E+08,  5.841542177435417E+08, -1.281381382777277E+07))* 1000 # m
v6 = np.array((-2.394184419030606E+01,  1.076395626919765E+01, -4.104233699513227E-02))*1000 # m/s
y0 = np.array((r1,r2,r3,r4,r5,r6,v1,v2,v3,v4,v5,v6))

In [ ]:
t_arr = np.linspace(0,Nsteps*dt,Nsteps)
orbits6 = RK4_solver(RHS_Orbital_motion,y0,dt,Nsteps)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111,projection='3d',)
ax.view_init(0,45)
ax.plot(orbits6[:,0,0], orbits6[:,0,1], orbits6[:,0,2], label="Sun")
ax.plot(orbits6[:,1,0], orbits6[:,1,1], orbits6[:,1,2], label="Jupiter")
ax.plot(orbits6[:,2,0], orbits6[:,2,1], orbits6[:,2,2], label="Ganymede")
ax.plot(orbits6[:,3,0], orbits6[:,3,1], orbits6[:,3,2], label="Callisto")
ax.plot(orbits6[:,4,0], orbits6[:,4,1], orbits6[:,4,2], label="Io")
ax.plot(orbits6[:,5,0], orbits6[:,5,1], orbits6[:,5,2], label="Europa")
ax.legend()

In [ ]:
ganorbit = orbits6[:,2] - orbits6[:,1]
callorbit = orbits6[:,3] - orbits6[:,1]
Ioorbit = orbits6[:,4] - orbits6[:,1]
Europaorbit = orbits6[:,5] - orbits6[:,1]
plt.plot(Europaorbit[:1000])

In [ ]:
fig = plt.figure(figsize=(20,20))
ax = fig.add_subplot(121,projection='3d',)
ax.plot(ganorbit[:,0],ganorbit[:,1],ganorbit[:,2], label="Ganymede's orbit")
ax.plot(callorbit[:,0],callorbit[:,1],callorbit[:,2], label="Callisto's orbit")
ax.plot(Ioorbit[:,0],Ioorbit[:,1],Ioorbit[:,2],label="Io's orbit")
ax.plot(Europaorbit[:,0],Europaorbit[:,1],Europaorbit[:,2],label="Europa's orbit")
ax.plot(0,0,0,   "o" , color='b', label="Jupiter")
ax.plot(ganorbit[0,0],ganorbit[0,1],ganorbit[0,2],  "o" , color='r')
ax.plot(callorbit[0,0],callorbit[0,1],callorbit[0,2],  "o" , color='r')
ax.plot(Ioorbit[0,0],Ioorbit[0,1],Ioorbit[0,2],  "o" , color='r')
ax.plot(Europaorbit[0,0],Europaorbit[0,1],Europaorbit[0,2],  "o" , color='r')
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("z")
ax.set_title("3-D plot of Europa's orbit")
ax.legend()
ax.view_init(20, 45)
ax1 = fig.add_subplot(122)
ax1.plot(ganorbit[:,0],ganorbit[:,1], label="Ganymede's orbit")
ax1.plot(callorbit[:,0],callorbit[:,1], label="Callisto's orbit")
ax1.plot(Ioorbit[:,0],Ioorbit[:,1],label="Io's orbit")
ax1.plot(Europaorbit[:,0],Europaorbit[:,1],label="Europa's orbit")
ax1.plot(0, 0,   "o" , color='b', label="Jupiter")
ax1.plot(ganorbit[0,0],ganorbit[0,1],  "o" , color='r')
ax1.plot(callorbit[0,0],callorbit[0,1],   "o" , color='r')
ax1.plot(Ioorbit[0,0],Ioorbit[0,1],  "o" , color='r')
ax1.plot(Europaorbit[0,0],Europaorbit[0,1],  "o" , color='r')
ax1.axis('scaled')
ax1.set_xlabel("x")
ax1.set_ylabel("y")
ax1.set_title("x-y plot of Europa's orbit")
ax1.legend()

In [ ]:
fig.savefig("Europa orbit.pdf")

In [ ]:
orbitdiff65 = orbits6[:,1]-orbits5[:,1]
orbitdiff64 = orbits6[:,1]-orbits4[:,1]
orbitdiff63 = orbits6[:,1]-orbits3[:,1]
orbitdiff62 = orbits6[:,1]-orbits2[:,1]
#plt.plot(orbitdiff62)
fig3 = plt.figure(figsize=(28,5))
ax1 = fig3.add_subplot(1,3,1)
#ax1.plot(orbits[:,0,0], orbits[:,0,1])
#ax1.plot(orbits[:,1,0], orbits[:,1,1])
ax1.plot(t_arr, orbitdiff62[:,0])
ax1.set_xlabel("Time")
ax1.set_ylabel("x")
ax1.grid()
ax1.set_title("Change in Jupiter's orbit in x direction")
ax2 = fig3.add_subplot(1,3,2)
ax2.plot(t_arr, orbitdiff62[:,1])
ax2.set_xlabel("Time")
ax2.set_ylabel("y")
ax2.set_title("Change in Jupiter's orbit in y direction")
ax2.legend()
ax2.grid()
ax3 = fig3.add_subplot(1,3,3)
ax3.plot(t_arr, orbitdiff62[:,2])
ax3.set_xlabel("Time")
ax3.set_ylabel("z")
ax3.set_title("Change in Jupiter's orbit in z direction")
ax3.legend()    
ax3.grid()

In [ ]:
fig3.savefig("4 moons effect on Jupiter.pdf")

In [ ]:
Energy, KE, PE = totalenergy(orbits6)
#print(Energy)
#plt.plot(t_arr, Energy[0])
#plt.plot(t_arr, Energy[1])
fig = plt.figure(figsize=(5,5))
ax = fig.add_subplot(111)
ax.plot(t_arr, Energy[0]+Energy[1]+Energy[2]+Energy[3]+Energy[4]+Energy[5])
ax.set_xlabel("Time")
ax.set_ylabel("E")
ax.set_title("Total Energy over time")

In [ ]:
fig.savefig("4 moon energy.pdf")

In [ ]:
L = totalangmomentum(orbits6)
angfig = plt.figure(figsize=(10,6), tight_layout=True)
#print(L[0,1])
ax1 = angfig.add_subplot(131)
ax1.plot(t_arr, L[:,0])
ax1.set_xlabel("time")
ax1.set_ylabel(r"$L_x$")
ax2 = angfig.add_subplot(132)
ax2.plot(t_arr, L[:,1])
ax2.set_xlabel("time")
ax2.set_ylabel(r"$L_y$")
ax3 = angfig.add_subplot(133)
ax3.plot(t_arr, L[:,2])
ax3.set_xlabel("time")
ax3.set_ylabel(r"$L_z$")
angfig.suptitle("Total Angular Momentum of system")

In [ ]:
angfig.savefig("4 moon ang momentum.pdf")

Due to Europa struggling to orbit without drifting I decided to plot Europa on its own:

In [ ]:
# Set up parameters
# Position and velocity values taken from https://ssd.jpl.nasa.gov/horizons/app.html#/ with date set to 06-03-24
Ndim = 3     #3D problem for now
Nsteps = 600000
Nbodies = 3
G = constants.G        # Appropriate units
dt = 60*30            # s
mSun = 1988500 * 10**24 # kg
mJup = 189818722 * 10**19     # kg
mGanymede = 1.482* 10**23     # kg from https://doi.org/10.1126%2Fscience.286.5437.77
mCallisto = 1.076* 10**23     # kg from https://doi.org/10.1126%2Fscience.286.5437.77
mIo = 0.8932*10**23           # kg from https://doi.org/10.1126%2Fscience.286.5437.77
mEuropa = 0.480*10**23        # kg from https://doi.org/10.1126%2Fscience.286.5437.77  
masses = np.array([mSun,mJup,mEuropa])
# Initial conditions
r1 = np.array((-1.142389309034058E+06,-5.009058423486830E+05,3.095836895255174E+04))*1000 # m
v1 = np.array((9.244118454845832E-03,-1.136017714077767E-02,-1.036700033153707E-04))*1000 # m/s
r2 = np.array((4.656502915513301E+08,5.834910884460863E+08,-1.283856850003496E+07))*1000 # m
v2 = np.array((-1.035714698991324E+01,8.770443233178732E+00,1.953703646066063E-01))*1000 # m/s
r3 = np.array((4.663298235072411E+08,5.826676629609896E+08 ,-1.286038673101431E+07))* 1000 # m
v3 = np.array((-1.954342380671196E+00, 1.571157393854946E+01, 5.784998944007400E-01))*1000 # m/s
r4 = np.array((4.646133092078045E+08,  5.850685313774981E+08, -1.280301715879852E+07))* 1000 # m
v4 = np.array((-1.722217212361970E+01,  4.327917320305715E+00, -3.556385749064983E-02))*1000 # m/s
r5 = np.array((4.652368585181826E+08,  5.835828494477390E+08, -1.284137882235295E+07))* 1000 # m
v5 = np.array((-1.410261135257421E+01, -8.071639586690274E+00, -4.515334237343565E-01))*1000 # m/s
r6 = np.array((4.657534133009982E+08,  5.841542177435417E+08, -1.281381382777277E+07))* 1000 # m
v6 = np.array((-2.394184419030606E+01,  1.076395626919765E+01, -4.104233699513227E-02))*1000 # m/s
y0 = np.array((r1,r2,r6,v1,v2,v6))

In [ ]:
t_arr = np.linspace(0,Nsteps*dt,Nsteps)
orbits7 = RK4_solver(RHS_Orbital_motion,y0,dt,Nsteps)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111,projection='3d',)
ax.view_init(0,45)
ax.plot(orbits7[:,0,0], orbits6[:,0,1], orbits6[:,0,2], label="Sun")
ax.plot(orbits7[:,1,0], orbits6[:,1,1], orbits6[:,1,2], label="Jupiter")
ax.plot(orbits7[:,2,0], orbits6[:,2,1], orbits6[:,2,2], label="Europa")
ax.legend()

In [ ]:
Europaorbit = orbits7[:,2] - orbits7[:,1]

In [ ]:
fig = plt.figure(figsize=(20,20))
ax = fig.add_subplot(121,projection='3d',)
ax.plot(Europaorbit[:,0],Europaorbit[:,1],Europaorbit[:,2],label="Europa's orbit")
ax.plot(0,0,0,   "o" , color='b', label="Jupiter")
ax.plot(Europaorbit[0,0],Europaorbit[0,1],Europaorbit[0,2],  "o" , color='r')
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("z")
ax.set_title("3-D plot of Europa's orbit")
ax.legend()
ax.view_init(20, 45)
ax1 = fig.add_subplot(122)
ax1.plot(Europaorbit[:,0],Europaorbit[:,1],label="Europa's orbit")
ax1.plot(0, 0,   "o" , color='b', label="Jupiter")
ax1.plot(Europaorbit[0,0],Europaorbit[0,1],  "o" , color='r')
ax1.axis('scaled')
ax1.set_xlabel("x")
ax1.set_ylabel("y")
ax1.set_title("x-y plot of Europa's orbit")
ax1.legend()

In [ ]:
fig.savefig("Europa solo orbit.pdf")

In [ ]:
orbitdiff72 = orbits7[:,1]-orbits2[:,1]
#plt.plot(orbitdiff62)
fig3 = plt.figure(figsize=(28,5))
ax1 = fig3.add_subplot(1,3,1)
#ax1.plot(orbits[:,0,0], orbits[:,0,1])
#ax1.plot(orbits[:,1,0], orbits[:,1,1])
ax1.plot(t_arr, orbitdiff72[:,0])
ax1.set_xlabel("Time")
ax1.set_ylabel("x")
ax1.grid()
ax1.set_title("Change in Jupiter's orbit in x direction")
ax2 = fig3.add_subplot(1,3,2)
ax2.plot(t_arr, orbitdiff72[:,1])
ax2.set_xlabel("Time")
ax2.set_ylabel("y")
ax2.set_title("Change in Jupiter's orbit in y direction")
ax2.legend()
ax2.grid()
ax3 = fig3.add_subplot(1,3,3)
ax3.plot(t_arr, orbitdiff72[:,2])
ax3.set_xlabel("Time")
ax3.set_ylabel("z")
ax3.set_title("Change in Jupiter's orbit in z direction")
ax3.legend()    
ax3.grid()

In [ ]:
fig3.savefig("Europa effect on Jupiter.pdf")